# Creating a NeuronUnit BluepyOpt optimisation with a reduced Cell model
such as the Izhikevich model



In [1]:
import matplotlib.pyplot as plt
plt.plot([1,0],[0,1])
plt.show()


<Figure size 640x480 with 1 Axes>

In [2]:
import bluepyopt as bpop
import pickle
import numpy as np
import numpy



from bluepyopt.ephys.models import ReducedCellModel
from neuronunit.optimisation.model_parameters import MODEL_PARAMS
from neuronunit.optimisation.optimization_management import test_all_objective_test, inject_and_plot_model, check_bin_vm30,check_bin_vm15

from neuronunit.optimisation.optimization_management import check_binary_match, three_step_protocol,inject_and_plot_passive_model

#from neuronunit.optimisation.optimization_management import check_bin_vm15


/home/user/anaconda3/lib/python3.7/site-packages/airspeed/__init__.py:505: FutureWarning: Possible nested set at position 8
  KEYVALSEP = re.compile(r'[ \t]*:[[ \t]*(.*)$', re.S)


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload

In [4]:
import bluepyopt.ephys as ephys


simple_cell = ephys.models.ReducedCellModel(
        name='simple_cell',
        params=MODEL_PARAMS["IZHI"],backend="IZHI")  
simple_cell.backend = "IZHI"


First we need to import the module that contains all the functionality to create electrical cell models

Setting up the cell
---------------------


In [5]:

from neuronunit.optimisation.model_parameters import MODEL_PARAMS
import numpy as np

simple_cell = ephys.models.ReducedCellModel(
        name='simple_cell',
        params=MODEL_PARAMS["IZHI"],backend="IZHI")  
simple_cell.backend = "IZHI"

In [6]:
model = simple_cell
model.params = {k:np.mean(v) for k,v in model.params.items() }

With this cell we can build a cell evaluator.

## Setting up a cell evaluator

To optimise the parameters of the cell we need to create cell evaluator object. 
This object will need to know which protocols to inject, which parameters to optimise, etc.

### Creating the protocols



### Running a protocol on a cell

Now we're at a stage where we can actually run a protocol on the cell. We first need to create a Simulator object.

The run() method of a protocol accepts a cell model, a set of parameter values and a simulator

Plotting the response traces is now easy:

### Defining eFeatures and objectives

For every response we need to define a set of eFeatures we will use for the fitness calculation later. We have to combine features together into objectives that will be used by the optimalisation algorithm. In this case we will create one objective per feature.

### In this case the features are derived from NeuronUnit test suites.

These features can either come from simulation, or from neuroelectro.

In [7]:



from neuronunit.optimisation.model_parameters import MODEL_PARAMS

MODEL_PARAMS["IZHI"].pop('C',None)
MODEL_PARAMS["IZHI"].pop('k',None)
MODEL_PARAMS["IZHI"].pop('vPeak',None)
MODEL_PARAMS["IZHI"].pop('vt',None)

#MODEL_PARAMS["IZHI"].pop('vr')#,None)#: <bluepyopt.parameters.Parameter at 0x7f26f22f2090>,
#MODEL_PARAMS["IZHI"].pop('a')# <bluepyopt.parameters.Parameter at 0x7f26f22f2110>,
#MODEL_PARAMS["IZHI"].pop('b')# <bluepyopt.parameters.Parameter at 0x7f26f22f2150>,
MODEL_PARAMS["IZHI"].pop('c')#<bluepyopt.parameters.Parameter at 0x7f26f22f20d0>,
MODEL_PARAMS["IZHI"].pop('d')#',None)# <bluepyopt.parameters.Parameter at 0x7f26f22f2190>}

'''
cnt = 0
lop={}
from bluepyopt.parameters import Parameter
not_frozen = []
for k,v in MODEL_PARAMS["IZHI"].items(): #["a","b","c","d"]:
    #v = MODEL_PARAMS["IZHI"][k]
    if cnt>len(MODEL_PARAMS["IZHI"])/4:
        
        p = Parameter(name=k,bounds=v,value=np.mean(v),frozen=True)

    else:
        not_frozen.append(k)
        p = Parameter(name=k,bounds=v,frozen=False)
    
        lop[k] = p
    cnt +=1
print('number of free params',len(lop))
'''


lop={}
from bluepyopt.parameters import Parameter
for k,v in MODEL_PARAMS["IZHI"].items():
    p = Parameter(name=k,bounds=v,frozen=False)
    lop[k] = p

print(lop)
simple_cell.params = lop

{'a': <bluepyopt.parameters.Parameter object at 0x7efbc3989c90>, 'b': <bluepyopt.parameters.Parameter object at 0x7efbc03c6e10>}


In [8]:

#lop.pop('C',None)
#del not_frozen[0]

In [9]:
lop

{'a': <bluepyopt.parameters.Parameter at 0x7efbc3989c90>,
 'b': <bluepyopt.parameters.Parameter at 0x7efbc03c6e10>}

In [10]:


simulated_experiment = True
features = None
allen = True
if simulated_experiment:
    while features is None:
        aug_nu_tests, OM, target = test_all_objective_test(lop.keys(),model_type="IZHI",protocol={'allen':False,'elephant':True})
        target = three_step_protocol(target)
        if hasattr(target,'everything'):
            if hasattr(target,'allen_15'):
                features = target.everything
                if allen:
                    simple_cell.allen = None
                    simple_cell.allen = True

                    tests = pickle.load(open("allen_NU_tests.p","rb"))
                    print('allen cell',tests[0].name)
                    nu_tests = tests[0]
                    for t in nu_tests:                    
                        if t.name in features.keys():
                            t.set_observation(features[t.name])
                            if t.observation['mean'] is not None:
                                t.observation['std'] = np.abs(np.mean(t.observation['mean']))
    
    #from sciunit.scores import ZScore
    #aug_nu_tests["RheobaseTest"].score_type = ZScore 
    #temp = list(aug_nu_tests.values())
    #temp.extend(nu_tests)
    #nu_tests = temp
    
    #nu_tests = list(nu_tests.values())
    #nu_tests = nu_tests_
        
if not allen:
    tests = pickle.load(open("processed_multicellular_constraints.p","rb"))
    nu_tests = tests['Hippocampus CA1 pyramidal cell'].tests
    from sciunit.scores import ZScore

    #nu_tests = list(nu_tests.values())
    nu_tests[0].score_type = ZScore 
    #for t in nu_tests:
     #   print(t.observation,t.name)
else:
    tests = pickle.load(open("allen_NU_tests.p","rb"))
    print('allen cell',tests[0].name)
    from random import choice
    nu_tests = choice(tests)
    #nu_tests = tests[2]
    tests_by_name = {t.name:t for t in nu_tests }


Random simulated data tests made
{'value': array(131.8359375) * pA}
{'value': array(2461.73711199) * ms}
{'mean': array(-65.22618636) * mV, 'std': array(1.42108547e-14) * mV}
{'value': array(25535378.42252496) * kg*m**2/(s**3*A**2)}
{'value': array(9.64049591e-08) * s**4*A**2/(kg*m**2)}
{'mean': array(0.00029) * s, 'std': array(0.) * s, 'n': 1}
{'value': array(44.80972298) * mV, 'mean': array(44.80972298) * mV, 'std': array(0.) * mV, 'n': 1}
{'mean': array(-23.67138404) * mV, 'std': array(0.) * mV, 'n': 1}


INFO:numexpr.utils:NumExpr defaulting to 4 threads.


allen cell 504615116
allen cell 504615116


In [11]:
#aug_nu_tests['RestingPotentialTest'].params
print(target.attrs)
simple_cell.params = target.attrs

{'C': 89.7960714285714, 'a': 0.0155454326944954, 'b': 14.39099242914141, 'c': -60, 'd': 10, 'k': 1.6, 'vPeak': 21.138338933982595, 'vr': -65.2261863636364, 'vt': -50, 'dt': 0.01, 'Iext': array([0., 0., 0., ..., 0., 0., 0.])}


In [12]:
#nu_tests = []
#nu_tests.append(tests_by_name['Spikecount_3.0x'])
#nu_tests.append(tests_by_name['Spikecount_1.5x'])
#tests_by_name['Spikecount_1.5x'].observation

In [13]:
#tests_by_name['Spikecount_3.0x'].observation
#tests_by_name['Spikecount_3.0x'].set_observation(tests_by_name['Spikecount_3.0x'].observation['mean'][0])
#tests_by_name['Spikecount_1.5x'].set_observation(tests_by_name['Spikecount_1.5x'].observation['mean'][0])

In [14]:


sweep_protocols = []
for protocol_name, _ in [('step1', 0.05)]:
    protocol = ephys.protocols.SweepProtocol(protocol_name, [None], [None])
    sweep_protocols.append(protocol)
twostep_protocol = ephys.protocols.SequenceProtocol('twostep', protocols=sweep_protocols)


In [15]:
lop

{'a': <bluepyopt.parameters.Parameter at 0x7efbc3989c90>,
 'b': <bluepyopt.parameters.Parameter at 0x7efbc03c6e10>}

In [16]:
nu_tests

### Creating the cell evaluator

We will need an object that can use these objective definitions to calculate the scores from a protocol response. This is called a ScoreCalculator.

In [17]:
simple_cell.params = lop
from nu_cell_evaluators import make_basic_objectives
from nu_cell_evaluators import make_allen_objectives
if allen:

    
    score_calc,objectives = make_allen_objectives(nu_tests,simple_cell)
else:
    
    score_calc,objectives = make_basic_objectives(nu_tests,simple_cell)


Combining everything together we have a CellEvaluator. The CellEvaluator constructor has a field 'parameter_names' which contains the (ordered) list of names of the parameters that are used as input (and will be fitted later on).

In [18]:
MODEL_PARAMS["IZHI"]
cell_evaluator = ephys.evaluators.CellEvaluator(
        cell_model=simple_cell,
        param_names=lop.keys(),
        fitness_protocols={twostep_protocol.name: twostep_protocol},
        fitness_calculator=score_calc,
        sim='euler')
simple_cell.params_by_names(lop.keys())
simple_cell.params;

### Evaluating the cell

The cell can now be evaluate for a certain set of parameter values.

## Setting up and running an optimisation

Now that we have a cell template and an evaluator for this cell, we can set up an optimisation.

In [19]:
MU=5
import dask
#from dask.distributed import Client

# Setup a local cluster.
# By default this sets up 1 worker per core
#client = Client()
#client.cluster
def dask_map_function(eval_,invalid_ind):
    results = []
    for x in invalid_ind:
        y = dask.delayed(eval_)(x)
        results.append(y)
    fitnesses = dask.compute(*results)
    return fitnesses



#        if self.selector_name == 'IBEA':
#            self.toolbox.register("select", tools.selIBEA)
#        elif self.selector_name == 'NSGA2':

#                 mutpb=1.0,
#                 cxpb=1.0,
optimisation = bpop.optimisations.DEAPOptimisation(
        evaluator=cell_evaluator,
        map_function = map,
        offspring_size = MU,
        selector_name='NSGA2',
        mutpb=1,
        cxpb=1)


And this optimisation can be run for a certain number of generations

In [ ]:

for k,v in hist.genealogy_history.items():
    ind_dict = cell_evaluator.param_dict(v)



The best individual found during the optimisation is the first individual of the hall of fame

In [ ]:
best_ind = hall_of_fame[0]

print('Fitness values: ', best_ind.fitness.values)
#best_ind = hall_of_fame[-1]

#print('Fitness values: ', best_ind.fitness.values)

We can evaluate this individual and make use of a convenience function of the cell evaluator to return us a dict of the parameters

In [ ]:
best_ind_dict = cell_evaluator.param_dict(best_ind)
objectives = cell_evaluator.evaluate_with_dicts(best_ind_dict)

In [ ]:
objectives['Spikecount_3.0x.Spikecount_3.0x']
objectives['Spikecount_1.5x.Spikecount_1.5x']


In [ ]:
model = cell_evaluator.cell_model
cell_evaluator.param_dict(best_ind)
model.attrs = {str(k):float(v) for k,v in cell_evaluator.param_dict(best_ind).items()}

opt = model.model_to_dtc()
opt.attrs = {str(k):float(v) for k,v in cell_evaluator.param_dict(best_ind).items()}
from neuronunit.optimisation.optimization_management import dtc_to_rheo, inject_and_plot_model30
opt = dtc_to_rheo(opt)
opt.rheobase
opt.attrs;

In [ ]:


#from neuronunit.tests.base import AMPL, DELAY, DURATION

#inject_and_plot_model(opt)






In [ ]:
vm301,vm151,_,_,target = inject_and_plot_model30(target)
vm302,vm152,_,_,opt = inject_and_plot_model30(opt)

5
#check_bin_vm30(target,opt)
#vm30,params,_,opt = inject_and_plot_model15(opt)


In [ ]:
opt

In [ ]:
target = three_step_protocol(target)

In [ ]:
opt = three_step_protocol(opt)


In [ ]:
print(opt.everything)

In [ ]:
print(opt.everything['Spikecount_1.5x'],target.everything['Spikecount_1.5x'])
print(opt.everything['Spikecount_3.0x'],target.everything['Spikecount_3.0x'])

In [ ]:
if simulated_experiment:

    check_binary_match(target,opt,plotly=False)
    check_binary_match(target,opt,snippets=False,plotly=False)
    inject_and_plot_passive_model(opt,second=target,plotly=False)
    inject_and_plot_passive_model(opt,second=target,plotly=True)



In [ ]:
opt.vm30,_,_,_,opt=inject_and_plot_model30(opt)
target.vm30,_,_,_,target=inject_and_plot_model30(target)
print(opt.rheobase)
target.vm15
check_bin_vm30(target,opt)
check_bin_vm15(target,opt)

opt.vm15



In [ ]:
print(opt.attrs)
print(target.attrs)

As you can see the evaluation returns the same values as the fitness values provided by the optimisation output. 
We can have a look at the responses now.

Let's have a look at the optimisation statistics.
We can plot the minimal score (sum of all objective scores) found in every optimisation. 
The optimisation algorithm uses negative fitness scores, so we actually have to look at the maximum values log.

In [ ]:
gen_numbers = logs.select('gen')
min_fitness = logs.select('min')
max_fitness = logs.select('max')
avg_fitness = logs.select('avg')

plt.clf()
plt.plot(gen_numbers, max_fitness, label='max fitness')
plt.plot(gen_numbers, avg_fitness, label='avg fitness')
plt.plot(gen_numbers, min_fitness, label='min fitness')

plt.xlabel('generation #')
plt.ylabel('score (# std)')
plt.legend()
plt.xlim(min(gen_numbers) - 1, max(gen_numbers) + 1) 
#plt.ylim(0.9*min(min_fitness), 1.1 * max(min_fitness)) 
plt.show()


In [ ]:
try:
    kvp_obj = {}
    for obj in objectives:
        kvp_obj[obj.name] = obj
    objectives = kvp_obj
except:
    pass

And this optimisation can be run for a certain number of generations

In [ ]:
import pandas as pd
import seaborn as sns
logbook = logs
#scores = [ m for m in logs ]
list_of_dicts = []
df1 = pd.DataFrame()
genes=[]
for _,v in hist.genealogy_history.items():
    genes.append(v.fitness.values)
for j,i in enumerate(objectives.keys()):
    index = i.split('.')[0]
    #print(len(pd.Series(genes).values))
    if j<len(genes):
        df1[str(index)] = pd.Series(genes).values[j]#, index=df1.index)


corr = df1.corr()
fig =plt.figure(figsize=(22,22))
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

In [ ]:
np.sum(np.sum(corr))/len(corr)

In [ ]:
np.sum(hist.genealogy_history[20].fitness.values)

In [ ]:
hist.genealogy_history[25].fitness

In [ ]:
corr

In [ ]:
genes=[]
min_per_generations = []
for i,v in hist.genealogy_history.items():
    if i%MU==0:
        min_per_gen = sorted([(gene,np.min(gene)) for gene in genes],key=lambda x: x[1])
        min_per_generations.append(min_per_gen[0][0])
        genes =[]
    genes.append(v.fitness.values)
df2 = pd.DataFrame()
for j,i in enumerate(objectives.keys()):
    if j<len(min_per_generations):
        df2[str(i)] = pd.Series(min_per_generations).values[j]#, index=df1.index)


In [ ]:
plt.clf()

for c in df2.columns:
    df2[c].plot()
plt.show()

In [ ]:
genes=[]

for _,v in hist.genealogy_history.items():
    genes.append(v.fitness.values)

In [ ]:
genes[10]

In [ ]:
plt.plot(range(0,len(genes)), genes, label='min fitness')


In [ ]:
for t in nu_tests:
    print(t.params)

In [ ]:
for t in nu_tests:
    print(dir(t.params))

##### 